In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import keras
from keras.models import Sequential
from keras.layers import Dense

data = pd.read_csv(r"C:\Users\Sanford Jone\Downloads\Bank_Personal_Loan_Modelling.csv")

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [3]:
data

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [4]:
data.corr()["Personal Loan"]

ID                   -0.024801
Age                  -0.007726
Experience           -0.007413
Income                0.502462
ZIP Code              0.000107
Family                0.061367
CCAvg                 0.366889
Education             0.136722
Mortgage              0.142095
Personal Loan         1.000000
Securities Account    0.021954
CD Account            0.316355
Online                0.006278
CreditCard            0.002802
Name: Personal Loan, dtype: float64

In [5]:
x = data.drop(['ID', 'ZIP Code', 'Personal Loan'], axis=1)
y = data['Personal Loan']

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x= scaler.fit_transform(x)

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

## PyTorch

In [8]:
from torch.utils.data import DataLoader, TensorDataset

In [9]:
class PyTorchModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1=nn.Linear(11,9).double()
        self.activ=nn.Sigmoid().double()
        self.fc2=nn.Linear(9,1).double()
        self.softmax=nn.Sigmoid()
        
    def forward(self,x):
        x=self.fc1(x)
        x=self.activ(x)
        x=self.fc2(x)
        x=self.softmax(x)
        return x
        

In [10]:
model=PyTorchModel()

In [11]:
model.parameters

<bound method Module.parameters of PyTorchModel(
  (fc1): Linear(in_features=11, out_features=9, bias=True)
  (activ): Sigmoid()
  (fc2): Linear(in_features=9, out_features=1, bias=True)
  (softmax): Sigmoid()
)>

In [12]:
x_train_t=torch.tensor(x_train)
x_test_t=torch.tensor(x_test)
y_train_t=torch.tensor(y_train.values)
y_test_t=torch.tensor(y_test.values)

In [13]:
criterion=nn.BCELoss()
optimiser=optim.SGD(model.parameters(),lr=0.005)

In [14]:
data_t=TensorDataset(x_train_t,y_train_t)

In [15]:
df=DataLoader(data_t,batch_size=32)

In [16]:
def binary_acc(y_pred,y):
    y_pred=torch.round(y_pred)
    crt_sum=(y_pred==y).sum().float()
    acc=crt_sum/len(y)
    acc=torch.round(acc*100)
    return acc

In [17]:
for i, j in df:
    pred = model(i)
    print(pred)

tensor([[0.5341],
        [0.5274],
        [0.5444],
        [0.5148],
        [0.5765],
        [0.5335],
        [0.4930],
        [0.5287],
        [0.5531],
        [0.5313],
        [0.5458],
        [0.5933],
        [0.5166],
        [0.5676],
        [0.5835],
        [0.5330],
        [0.5280],
        [0.5984],
        [0.4969],
        [0.5196],
        [0.5763],
        [0.5413],
        [0.5093],
        [0.6002],
        [0.5262],
        [0.5030],
        [0.5425],
        [0.5652],
        [0.5304],
        [0.5331],
        [0.5233],
        [0.5282]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
tensor([[0.5376],
        [0.5216],
        [0.5289],
        [0.5504],
        [0.5306],
        [0.5061],
        [0.5512],
        [0.5654],
        [0.5123],
        [0.5337],
        [0.5380],
        [0.5638],
        [0.5461],
        [0.5423],
        [0.5215],
        [0.5152],
        [0.5058],
        [0.5321],
        [0.5566],
        [0.5013],
        [0.553

In [18]:
def train(model,epoch,data):
    for i in range(epoch):
        acc_epoch=0
        for x,y in data:
            pred=model(x)
#             print(pred.shape, k.shape)
            pred = torch.reshape(pred, [y.shape[-1]])
            y=y.double()
            
            loss=criterion(pred,y)
            loss.backward()
            #update
            optimiser.step()
            optimiser.zero_grad()
            
            acc=binary_acc(pred.unsqueeze(1),y.unsqueeze(1))
            
            acc_epoch+=acc.item()
            
        if i%10==0 :
            print(f"Epoch: {i+1}, Error: {loss}, Accuracy: {acc_epoch/len(data):.3f}")
            
    

In [19]:
train(model,200,df)

Epoch: 1, Error: 0.5158745192773952, Accuracy: 78.936
Epoch: 11, Error: 0.26982118376712977, Accuracy: 90.964
Epoch: 21, Error: 0.25388773974122697, Accuracy: 90.964
Epoch: 31, Error: 0.24279387022519824, Accuracy: 90.964
Epoch: 41, Error: 0.23295286037382856, Accuracy: 90.964
Epoch: 51, Error: 0.22405082648777572, Accuracy: 90.964
Epoch: 61, Error: 0.21601601095794118, Accuracy: 90.964
Epoch: 71, Error: 0.20879179890001984, Accuracy: 90.964
Epoch: 81, Error: 0.2023130883722918, Accuracy: 91.109
Epoch: 91, Error: 0.19651114691323748, Accuracy: 91.564
Epoch: 101, Error: 0.1913209596611738, Accuracy: 91.891
Epoch: 111, Error: 0.18668573869778113, Accuracy: 92.636
Epoch: 121, Error: 0.18255830573142892, Accuracy: 92.945
Epoch: 131, Error: 0.17890031985706242, Accuracy: 93.127
Epoch: 141, Error: 0.17568032558093097, Accuracy: 93.300
Epoch: 151, Error: 0.17287127899194232, Accuracy: 93.573
Epoch: 161, Error: 0.17044793909593461, Accuracy: 93.664
Epoch: 171, Error: 0.16838438005886844, Accur

In [20]:
test_df = TensorDataset(x_test_t,y_test_t)
test_data = DataLoader(test_df, batch_size = 1)

In [21]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for x,y in test_data:
        pred = model(x)
        pred = torch.round(pred)
        y_pred_list.append(pred.item())

In [22]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      1343
           1       0.89      0.48      0.62       157

    accuracy                           0.94      1500
   macro avg       0.92      0.74      0.79      1500
weighted avg       0.94      0.94      0.93      1500



## Keras

In [23]:
model = Sequential()
model.add(Dense(64, input_dim=31, activation='relu',input_shape=(x_train.shape[-1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
train=model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test), verbose=1)

Epoch 1/50
110/110 [==============================] - 3s 7ms/step - loss: 0.4937 - accuracy: 0.7997 - val_loss: 0.3494 - val_accuracy: 0.8953
Epoch 2/50
110/110 [==============================] - 1s 6ms/step - loss: 0.2906 - accuracy: 0.9077 - val_loss: 0.2766 - val_accuracy: 0.8953
Epoch 3/50
110/110 [==============================] - 1s 6ms/step - loss: 0.2411 - accuracy: 0.9080 - val_loss: 0.2380 - val_accuracy: 0.8960
Epoch 4/50
110/110 [==============================] - 1s 6ms/step - loss: 0.2114 - accuracy: 0.9131 - val_loss: 0.2095 - val_accuracy: 0.9047
Epoch 5/50
110/110 [==============================] - 1s 7ms/step - loss: 0.1897 - accuracy: 0.9243 - val_loss: 0.1884 - val_accuracy: 0.9253
Epoch 6/50
110/110 [==============================] - 1s 6ms/step - loss: 0.1738 - accuracy: 0.9349 - val_loss: 0.1726 - val_accuracy: 0.9347
Epoch 7/50
110/110 [==============================] - 1s 7ms/step - loss: 0.1614 - accuracy: 0.9409 - val_loss: 0.1599 - val_accuracy: 0.9380
Epoch 

In [27]:
ypred = model.predict(x_test).round()

47/47 [==============================] - 0s 2ms/step


In [28]:
from sklearn.metrics import classification_report 

In [30]:
print(classification_report(y_test,ypred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1343
           1       0.93      0.82      0.87       157

    accuracy                           0.97      1500
   macro avg       0.96      0.90      0.93      1500
weighted avg       0.97      0.97      0.97      1500

